## 들리는대로 일단 쓰고 나중에 키워드 위주로 다시 공부하자....

- 두가지의 확률이 동시에 일어났을때 => 조인트(joint propability) 라고 부른다.
    - 조인트에는 어떤 문제가 있다고 한다...? 전체 케이스를 모두 알 수 없다ㅏ..?
    - 결합확률은 알기가 어렵다. 
- 그래서 베이지안은 조건부확률을 계산한다. ()
    - 개별확률과 결합 확률을 알면 구할 수 있다. 
        - 개별확률(Total Propability)은 Summing out이라고도 한다. = Marginalization
        - $P(A)=\sum_B{P(A,B)}$
- 우리는 N-gram 모델로 띄어쓰기를 예측할 때 조건부 확률을 사용했던 적이 있다.

- $f^*=argmin_fP(f(x) \neq Y)$ :에러가 가장 작은 함수 를 이렇게 표현할 것이다.
    - optimal classifier
    - MLE로 추정함
    - 1 - argmax 어쩌구 식과 같다.


- logistic regression이 lenear regression보다 좋다(?)
    - bayes risk 때문에
    - 

- 조건부 독립(Conditional independance)
    - 모든 콤포지션을 구할 수 가 없으므로 bayes 가정을 함 (메모리 초과)
        - 그렇다고 데이터를 줄여? 댓츠 노우노우
    - 문제가 있음 -> 실제 월드에서는 피쳐들 간에 강한 디펜던시가 있는 경우가 있음.
    - 그럼에도 불구하고 계산을 위해서 가정을 한다.
    - bag of words도, vs model도 각각 다 독립이라고 가정했었다. 얘도 그대로 가져다 쓸 수 있다.
    - 반대?: Marginal independence

- $\hat{P}(c)$ 는 사전 지식  
$\prod$는 곱집합, $\times$

- log를 씌워도 결과에 차이가 없다....monotonic...(단조 증가)....
    - 우리는 대소 관계만 볼거니깡...?

- $\hat{P}(t|c) = \frac{T_{ct}}{\sum_{t' \in V}{T_{ct'}}}$
    - 여기서 V는 모든 feature의 갯수? feature의 범위? 라고 했던거 같다..

- 라플라스 스무딩 = Add-one smoothing
    - 무조건 최초 한번은 관측이 되었다고 가정을 한다.
    - 최소 단어의 기본 확률값을 반영하는 개념

# 오늘 공부한 내용
- https://nlp.stanford.edu/IR-book/newslides.html
- 13. Text classification & Naive Bayes (ppt2)

# 코딩
- ppt의 33~34p 실습
- 숫자 주석은 ppt의 라인번호임
- 라인번호와 대조하면서 복습하기!

### 33p. traning

In [1]:
# Documnets
# ?는 맞춰야하는 타겟
D = [
    (1, "Chinese Beijing Chinese", "yes"),
    (2, "Chinese Chinese Shanghai", "yes"),
    (3, "Chinese Macao", "yes"),
    (4, "Tokyo Japan Chinese", "no"),
    (5, "Chinese Chinese Chinese Tokyo Japan", "?")
]

# Class
C = ["yes", "no"]

In [41]:
def training(C, D):
    V = list()
    prior = list(0.0 for _ in range(len(C)))
    for d in map(lambda d:d[1], D):
        for t in d.lower().split():
#             if t not in V:
            V.append(t)
    # if문 거치는 것 보다 set이 더 빠르다.
    V = list(set(V)) # 1 
    condprob = list(list(0.0 for _ in range(len(C)))
                   for _ in range(len(V)))
    trainingData = list(filter(lambda d:d[-1] in C, D))
    testingData = list(filter(lambda d:d[-1] not in C, D))
    N = len(trainingData) # 2
    for c in C: # 3
        Dc = list(filter(lambda d:d[-1] == c, 
                         trainingData))
        Nc = len(Dc) # 4
        prior[C.index(c)] = Nc/N # 5
        textc = " ".join(map(lambda d:d[1], Dc)) # 6
        Tct = list(0 for _ in range(len(V)))
        for t in V: # 7
            Tct[V.index(t)] = len(list(
                filter(lambda x: t==x, 
                             textc.lower().split()))) # 8
        for t in V: # 9
#             print(Tct)
            condprob[V.index(t)][C.index(c)] =\
                        (Tct[V.index(t)]+1)/(sum(Tct)+len(V))
            # 10
    return V, prior, condprob # 11

In [42]:
V, prior, condprob = training(C, D)

### 34p. testing

In [44]:
from math import log
def apply(C, V, prior, condprob, d):
    score = list(0.0 for _ in range(len(C)))
    W = list(filter(lambda t:t in V,
                   d.lower().split())) # 1
    for c in C: # 2
        score[C.index(c)] = log(prior[C.index(c)]) # 3
        for t in W: # 4
            score[C.index(c)] += \
            log(condprob[V.index(t)][C.index(c)]) # 5
    return C[0] if score[0] > score[1] else C[1], score

In [45]:
predictY, prob = apply(C, V, prior, condprob, D[-1][1])

In [47]:
from math import exp
predictY, list(map(lambda p:exp(p), prob))

('yes', [0.00030121377997263, 0.00013548070246744215])

In [48]:
d = "Korea Chinese"

predictY, prob = apply(C, V, prior, condprob, d)
predictY, list(map(lambda p:exp(p), prob))

('yes', [0.3214285714285714, 0.055555555555555566])